In [ ]:
open System.IO

let lines = File.ReadAllLines("./out/passport-1-words.txt")

let clearLines =
    lines
    |> Seq.map(fun m -> m.ToLower())
    |> Seq.filter(fun m -> m.Length > 1)

let wordsLine = String.Join(" ", clearLines)
wordsLine


отделением уфмс россии видам по челябинской области гор. трехгорный сл дата выдачи 23.12.2013 код подразделения 740-049 код м. анчная подпись 0-049 путилов максим сл отчество александрович дата муж. рождения 11.03.1980 mecro рождения гор. златоуст челябинской обл. pnrusputilov<<maksim<aleksandrovi3<<< 7513680817rus8003111m<<<<<<<3131223740049<50

### Passport parse

In [ ]:
open System
open System.Text.RegularExpressions

let inline iif c t f = if c then t else f
let inline iif2 (c1, t1) (c2, t2) f = if c1 then t1 else (if c2 then t2 else f)
let inline iin c t = iif c t null

let iim (m: Match) = iin m.Success m.Value
let iimg (m: Match) i = iin (m.Groups.Count > i) m.Groups.[i].Value 
let iimc (m: MatchCollection) i = iin (m.Count > i) m.[i].Value 
let iimcg (m: MatchCollection) i j = iin (m.Count > i && m.[i].Groups.Count > j) m.[i].Groups.[j].Captures.[0].Value 
let iimlg (m: MatchCollection) j = iimcg m (m.Count - 1) j

let toNullable = (function | Some x -> x | None -> null)

In [ ]:
open System.IO

let toClean (str: string) = str.ToLowerInvariant().Replace("\"", "")
let toCleans = Seq.map toClean

let firstNames = File.ReadAllLines("./data/fnames.csv") |> toCleans
let lastNames = File.ReadAllLines("./data/lnames.csv") |> toCleans
let midNames = File.ReadAllLines("./data/mnames.csv") |> toCleans

let isSomeName c str = c |> Seq.contains str
let findSomeName c = Seq.tryFind (isSomeName c) >> toNullable

let findFirstName x = x |> findSomeName firstNames
let findLastName x = x |> findSomeName lastNames
let findMiddleName x = x |> findSomeName midNames


In [ ]:

let issuer = Regex "(^|федерация)(.*?)(дата)"
let issuerCode = Regex "\d{3}\-\d{3}"
let date = Regex "\d{2}\.\d{2}\.\d{4}"
let birthPlace = Regex "(рожден.*?\s+)(.*?)([a-z]+)"
let number = Regex "\d{9}"
let sex = Regex "муж\.|жен\."

let parsePassportRf str = 

    let issuerMatch = issuer.Match str
    let issuerCodeMatch = issuerCode.Match str
    let dateMatch = date.Matches str
    let birthPlaceMatch = birthPlace.Matches str
    let numberMatch = number.Match str
    let sexMatch = sex.Match str

    let sexResult = iim sexMatch

    let words = str.Split " "

    let firstName = findFirstName words

    {|
        Issuer = iimg issuerMatch 2
        IssueDate = iimc dateMatch 0
        issuerCode = iim issuerCodeMatch
        BirthDate = iimc dateMatch 1
        BirthPlace = iimlg birthPlaceMatch 2
        Number = iim numberMatch
        FirstName = findFirstName words
        LastName = findLastName words
        MiddleName = findMiddleName words
        Sex = (iif2 (sexResult = "муж.", "male") (sexResult = "жен.", "female") null) 
    |}    
        

parsePassportRf wordsLine


 

BirthDate,BirthPlace,FirstName,IssueDate,Issuer,LastName,MiddleName,Number,Sex,issuerCode
11.03.1980,гор. златоуст челябинской обл.,максим,23.12.2013,отделением уфмс россии видам по челябинской области гор. трехгорный сл,по,александрович,751368081,male,740-049
